### US data processing v. 2024



In [ ]:
### Imports 
import os
import numpy as np
from functions import *
from numba import cuda
import json

In [ ]:
# Global Parameter dictionary

prms = dict(
    window = 42, # Window size (in pixels)
    stride = 14, # By how many pixels we move between final correlation data points
    max_disp = 10, # Maximal displacement (in pixels) allowed between two frames
    left = 0, # If you want to left trim your speckles 
    right = 540, # Right trim for your speckles (formerly `pos_rotor`, but now we are also trimming maps beyond it) 
    nchan_bf = 17, # Number of channels on which you do the beamforming ; note : it is one-sided (we will do +nchan above and +nchan below)
    min_hil = 0.004*2048, # Minimum hilbert (think US) amplitude needed to validate a velocity point
    min_corr_score = 0.65, # Minimum correlation amplitude between two signals to validate a velocity point
    # do_cuda = cuda.is_available(),
    do_cuda = False,
    folder = 'Data',
    acq_prefix = 'gpt_',
    ref_path = None,
    calib_path = 'Data/gpt_val.json',
    redo_ref = False,
    redo_bf = False,
    redo_process = False
    )

In [ ]:
# Find all relevant acquisitions
acq_paths = list(os.walk(prms['folder']))[0][1]
acq_paths = [prms['folder'] + '/' + acq for acq in acq_paths if prms['acq_prefix'] in acq] # Subfolders of the main folder

### Main programme
for acq_path in acq_paths:
    
    if prms['ref_path'] is None: 
        ref_path = acq_path # Allows you to do "self-references"

    # Fetching all files, updating config and displaying a few things
    files = find_files(acq_path)
    config = load_config(acq_path + '/config.mat')
    ref_config = load_config(ref_path + '/config.mat')
    config = update_config(config, prms, save_path=acq_path, ref_path=ref_path)
    print_details(acq_path, config)

    # # Processing 
    ref = make_ref(ref_path, ref_config, recompute=prms['redo_ref'])
    bf_files = beamform(files, config, ref, recompute=prms['redo_bf'])
    hil_all, disp_all, score_all = process(bf_files, config, recompute=prms['redo_process'])

    # Validating and saving data
    disp_valid = disp_all.copy()
    too_weak = (0.5*(hil_all[:,:-1,:] + hil_all[:,1:,:]) < config['min_hil']) 
    not_correlated = (score_all < config['min_corr_score'])
    disp_valid[too_weak | not_correlated] = np.nan

    # Compute velocity
    velocity = np.nan * np.ones_like(disp_valid)
    r_true = np.nan * np.ones(velocity.shape[-1]) 
    if prms['calib_path'] is not None:
        with open(prms['calib_path']) as myfile:
            calib = json.load(myfile)
        r_true, velocity, _, _, _ = calibrate_one(calib, config, disp=disp_valid)

    # Save results
    np.savez(acq_path + '/processed', r_true=r_true, hil=hil_all, disp=disp_valid, velocity=velocity, ref=ref, score=score_all)